In [1]:
%pip install 'vanna[chromadb,openai,postgres]'

Note: you may need to restart the kernel to use updated packages.


In [ ]:
from vanna.openai import OpenAI_Chat
from vanna.chromadb import ChromaDB_VectorStore
from openai import OpenAI
import os
from dotenv import load_dotenv

from contextlib import redirect_stdout, redirect_stderr
import io

## You will need to have the OpenAI key in the .env file, along with having it set to NEW_OPENAI_API_KEY=sk-....

In [ ]:
# Load the .env file
load_dotenv()

# Retrieve the values using os.environ
api_key = os.getenv('NEW_OPENAI_API_KEY')

In [ ]:
print(f"The value of the api_key is the {api_key}")

## Running this code will generate a chroma.sqlite3 file, which is likely the ChromaDB?

The code below creates a chroma.sqlite3 file

In [ ]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

# Running this code creates the chroma.sqlite3 file
vn = MyVanna(config={'api_key': api_key, 'model': 'gpt-3.5-turbo'})

In [ ]:
import requests
ip = requests.get('https://api.ipify.org').text
print(f"Your Colab runtime IP is: {ip}")

## Remember the host number might change! Change that whenever the VM stops running

A sample of what the database looks like is in the csv of this code or in /Vanna/courses.csv

In [ ]:
vn.connect_to_postgres(host='34.134.126.254', dbname='tutorialDB', user='postgres', password='butlar', port='5432')

## Running this code will create the information schema (Directories full of training data and embeddings?)

In [ ]:
# The information schema query may need some tweaking depending on your database. This is a good starting point.

# This SQL code retrieves all the information across all the tables in all tables within a database.
df_information_schema = vn.run_sql("SELECT * FROM INFORMATION_SCHEMA.COLUMNS")

# This will break up the information schema into bite-sized chunks that can be referenced by the LLM
plan = vn.get_training_plan_generic(df_information_schema)

In [ ]:
vn.train(plan=plan)

In [ ]:
vn.train(documentation="My tables outline the coures for a college university. In each row you can see the college" \
"course with its respective course number and name, which professor teaches it, what times it runs from," \
"where it meets, how many students can attend, and how many units the course is worth. ")

In [ ]:
vn.train(documentation="The descr marks the course description. The course itself can be referenced as ENG EC or just EC. As an example" \
"the course description for ENG EC 520 is Digital Image Processing and Communication and it meets in building EPC in room 208.")

In [ ]:
# At any time you can inspect what training data the package is able to reference
training_data = vn.get_training_data()
training_data

In [ ]:
vn.train(documentation="The ENG identifier in the college column stands for Engineering. However, " \
"anytime it a course is referenced as EC 311 for example, ENG is implied. So the whole course" \
"code for EC 311 would be ENG EC 311.")

In [ ]:
vn.train(documentation="For the locations, PHO stands for Photonics. When a question liked which classes are taught in Photonics or photonics," \
"this question is asking which courses are located in PHO?")

In [ ]:
# You can remove training data if there's obsolete/incorrect information.
# vn.remove_training_data(id='5dc9b25e-6409-51b3-9ae7-1e51152309b8-doc')
# training_data

In [ ]:
# %%capture captured_output
my_question = "What is the course description of EC 522?"
response = vn.ask(question=my_question, allow_llm_to_see_data=True)

In [ ]:
capt_vanna_ans= captured_output.stdout
if "ERROR" not in capt_vanna_ans:
    lines = capt_vanna_ans.splitlines()

    SQL_query_commands = ["SELECT", "AND", "WHERE", "FROM", ";", "gpt", "SQL", "LLM", "`"]
    filtered_lines = [line for line in lines if not any(substring in line for substring in SQL_query_commands)]
    vanna_ans_tables = "\n".join(filtered_lines)

    print(vanna_ans_tables)
else:
    print("Bruh happened")

In [ ]:
client = OpenAI()
client.api_key = api_key

completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "developer", "content": "You are a helpful assistant in interpreting data tables into complete sentences and an intelligible response."},
    {"role": "user", "content": f"Answer the question of: {my_question}, given this data table of{vanna_ans_tables}"}
  ]
)

print(completion.choices[0].message.content)


In [ ]:
class MyVanna(ChromaDB_VectorStore, OpenAI_Chat):
    def __init__(self, config=None):
        ChromaDB_VectorStore.__init__(self, config=config)
        OpenAI_Chat.__init__(self, config=config)

def answer_course_question_new(question: str):
    vn = MyVanna(config={'api_key': api_key, 'model': 'gpt-3.5-turbo'})
    vn.connect_to_postgres(host='34.134.126.254', dbname='tutorialDB', user='postgres', password='butlar', port='5432')
    # training_data = vn.get_training_data()
    # print(training_data)

    dummy_output = io.StringIO()
    dummy_error = io.StringIO()
    with redirect_stdout(dummy_output), redirect_stderr(dummy_error):
        response = vn.ask(question=question, allow_llm_to_see_data=True, print_results=False)
    
    if response[1]:
        print(f"The response is {response[1]}")
    return response[1]

newple = answer_course_question_new("What courses does Tali Moreshet teach?")

In [ ]:
print(newple[1])

In [ ]:
from vanna.flask import VannaFlaskApp
app = VannaFlaskApp(vn)
app.run()